In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [8]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')
ExchangeAPI = userdata.get('ExchangeAPI')

In [25]:
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """Gets the conversion factor between two currencies."""
  url = f'https://v6.exchangerate-api.com/v6/{ExchangeAPI}/pair/{base_currency}/{target_currency}'
  response = requests.get(url)
  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
    """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """
    return base_currency_value * conversion_rate


In [10]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1762041601,
 'time_last_update_utc': 'Sun, 02 Nov 2025 00:00:01 +0000',
 'time_next_update_unix': 1762128001,
 'time_next_update_utc': 'Mon, 03 Nov 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 88.7594}

In [26]:
llm = ChatGoogleGenerativeAI(
    model = 'gemini-2.5-pro',
    google_api_key=GOOGLE_API_KEY
)
llm_with_tools = llm.bind_tools([convert, get_conversion_factor])

In [27]:
messages = []
query = HumanMessage("What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd")
messages = [query]
ai_message = llm_with_tools.invoke(messages)
messages.append(ai_message)

In [28]:
ai_message.tool_calls

[{'name': 'convert',
  'args': {'base_currency_value': 10},
  'id': '477d51ad-fe06-40be-aaa4-c3bb36e873f9',
  'type': 'tool_call'},
 {'name': 'get_conversion_factor',
  'args': {'target_currency': 'USD', 'base_currency': 'INR'},
  'id': '9cef0521-7d82-42c1-8a3f-46063e6d29d6',
  'type': 'tool_call'}]

In [29]:
import json

for tool_call in ai_message.tool_calls:
  if tool_call['name']=='get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    messages.append(tool_message1)
  if tool_call['name']=='convert':
    tool_call['args']['conversion_rate']=conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)


In [30]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_conversion_factor', 'arguments': '{"target_currency": "USD", "base_currency": "INR"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-pro', 'safety_ratings': []}, id='run--3b48763a-8de8-4a2e-bf8b-78537c67689d-0', tool_calls=[{'name': 'convert', 'args': {'base_currency_value': 10, 'conversion_rate': 0.01127}, 'id': '477d51ad-fe06-40be-aaa4-c3bb36e873f9', 'type': 'tool_call'}, {'name': 'get_conversion_factor', 'args': {'target_currency': 'USD', 'base_currency': 'INR'}, 'id': '9cef0521-7d82-42c1-8a3f-46063e6d29d6', 'type': 'tool_call'}], usage_metadata={'input_tokens': 142, 'output_tokens': 810, 'total_tokens': 952, 'input_token_details': {'cache_read': 0}, 'output_

In [31]:
llm_with_tools.invoke(messages).content

'The conversion factor from INR to USD is 0.01127.\n10 INR is equal to 0.1127 USD.'